In [1]:
import asyncio

from joblib import Parallel, delayed

import requests as req
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

import time
import multiprocessing as mp

import pandas as pd
pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

from tqdm.notebook import tqdm

#from webdriver_manager.chrome import ChromeDriverManager
#PATH=ChromeDriverManager().install()

PATH=r'Chromedriver\chromedriver'

import warnings
warnings.filterwarnings('ignore')

# stats = stats_first_clean = stats_chess.com

In [2]:
stats_original = pd.read_csv(r'..\data\stats_first_clean.csv')
stats = stats_original.copy()
stats.head()

,Unnamed: 0,name,username,profile,user_id,title,fide,country,current_rapid_rating,highest_rapid_rating,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,bullet_win,bullet_draw,bullet_loss
0,0,"Engine, Komodo",komodochess,https://www.chess.com/member/KomodoChess,24944922,GM,3411.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Plát, Vojtěch",vojtechplat,https://www.chess.com/member/VojtechPlat,37712368,GM,2950.0,Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0,2661.0,2750.0,32.0,8.0,47.0
2,2,"Carlsen, PlayMagnus",playmagnus,https://www.chess.com/member/PlayMagnus,19578862,GM,2882.0,Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Carlsen, Magnus",magnuscarlsen,https://www.chess.com/member/MagnusCarlsen,3889224,GM,2882.0,Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
4,4,"Caruana, Fabiano",fabianocaruana,https://www.chess.com/member/FabianoCaruana,11177810,GM,2835.0,United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0


In [3]:
stats.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1508 non-null   int64  
 1   name                   1241 non-null   object 
 2   username               1508 non-null   object 
 3   profile                1508 non-null   object 
 4   user_id                1508 non-null   int64  
 5   title                  1508 non-null   object 
 6   fide                   913 non-null    float64
 7   country                1508 non-null   object 
 8   current_rapid_rating   1055 non-null   float64
 9   highest_rapid_rating   997 non-null    float64
 10  rapid_win              1055 non-null   float64
 11  rapid_draw             1055 non-null   float64
 12  rapid_loss             1055 non-null   float64
 13  current_blitz_rating   1388 non-null   float64
 14  highest_blitz_rating   1382 non-null   float64
 15  blit

In [4]:
stats.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,1508.0,7.535000e+02,4.354664e+02,0.0,376.75,753.5,1130.25,1507.0
user_id,1508.0,4.145837e+07,4.256272e+07,3453.0,15665282.00,29307905.0,55585392.00,299430199.0
fide,913.0,2.002938e+03,1.061741e+03,0.0,2405.00,2523.0,2600.00,3411.0
current_rapid_rating,1055.0,2.382518e+03,3.165229e+02,263.0,2306.00,2455.0,2567.50,2948.0
highest_rapid_rating,997.0,2.555735e+03,2.535907e+02,1200.0,2470.00,2590.0,2715.00,3045.0
rapid_win,1055.0,3.567299e+01,8.903462e+01,0.0,4.00,12.0,31.00,1561.0
rapid_draw,1055.0,1.287014e+01,4.168809e+01,0.0,1.00,4.0,12.00,1150.0
rapid_loss,1055.0,1.578389e+01,4.330288e+01,0.0,2.00,6.0,16.00,878.0
current_blitz_rating,1388.0,2.696158e+03,2.428279e+02,255.0,2565.00,2725.0,2860.25,3287.0
highest_blitz_rating,1382.0,2.797418e+03,2.223825e+02,1265.0,2680.25,2824.5,2948.00,3332.0


In [5]:
# drop profile and user_id
stats = stats.drop(['profile', 'user_id', 'username', 'title', 'fide'], axis=1)
stats

,Unnamed: 0,name,country,current_rapid_rating,highest_rapid_rating,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,bullet_win,bullet_draw,bullet_loss
0,0,"Engine, Komodo",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Plát, Vojtěch",Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0,2661.0,2750.0,32.0,8.0,47.0
2,2,"Carlsen, PlayMagnus",Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Carlsen, Magnus",Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
4,4,"Caruana, Fabiano",United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,1503,"Zvonko, Stanojoski",North Macedonia,NaN,NaN,NaN,NaN,NaN,2375.0,2430.0,50.0,9.0,27.0,2102.0,2083.0,13.0,1.0,7.0
1504,1504,"Gurevich, Dmitry",United States,2375.0,2555.0,7.0,2.0,6.0,2415.0,2675.0,3293.0,538.0,2521.0,2400.0,2594.0,496.0,47.0,402.0
1505,1505,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2961.0,2967.0,338.0,22.0,96.0,NaN,NaN,NaN,NaN,NaN
1506,1506,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2771.0,2763.0,138.0,3.0,11.0,NaN,NaN,NaN,NaN,NaN


In [6]:
stats.columns = stats.columns.str.capitalize()
stats

,Unnamed: 0,Name,Country,Current_rapid_rating,Highest_rapid_rating,Rapid_win,Rapid_draw,Rapid_loss,Current_blitz_rating,Highest_blitz_rating,Blitz_win,Blitz_draw,Blitz_loss,Current_bullet_rating,Highest_bullet_rating,Bullet_win,Bullet_draw,Bullet_loss
0,0,"Engine, Komodo",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Plát, Vojtěch",Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0,2661.0,2750.0,32.0,8.0,47.0
2,2,"Carlsen, PlayMagnus",Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Carlsen, Magnus",Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
4,4,"Caruana, Fabiano",United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,1503,"Zvonko, Stanojoski",North Macedonia,NaN,NaN,NaN,NaN,NaN,2375.0,2430.0,50.0,9.0,27.0,2102.0,2083.0,13.0,1.0,7.0
1504,1504,"Gurevich, Dmitry",United States,2375.0,2555.0,7.0,2.0,6.0,2415.0,2675.0,3293.0,538.0,2521.0,2400.0,2594.0,496.0,47.0,402.0
1505,1505,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2961.0,2967.0,338.0,22.0,96.0,NaN,NaN,NaN,NaN,NaN
1506,1506,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2771.0,2763.0,138.0,3.0,11.0,NaN,NaN,NaN,NaN,NaN


In [7]:
# drop bullet columns, it's irrevelant for our study
stats = stats.drop(['Current_bullet_rating', 'Highest_bullet_rating', 'Bullet_win', 'Bullet_draw', 'Bullet_loss', 'Unnamed: 0'], axis=1)
stats

,Name,Country,Current_rapid_rating,Highest_rapid_rating,Rapid_win,Rapid_draw,Rapid_loss,Current_blitz_rating,Highest_blitz_rating,Blitz_win,Blitz_draw,Blitz_loss
0,"Engine, Komodo",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Plát, Vojtěch",Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0
2,"Carlsen, PlayMagnus",Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Carlsen, Magnus",Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0
4,"Caruana, Fabiano",United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1503,"Zvonko, Stanojoski",North Macedonia,NaN,NaN,NaN,NaN,NaN,2375.0,2430.0,50.0,9.0,27.0
1504,"Gurevich, Dmitry",United States,2375.0,2555.0,7.0,2.0,6.0,2415.0,2675.0,3293.0,538.0,2521.0
1505,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2961.0,2967.0,338.0,22.0,96.0
1506,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2771.0,2763.0,138.0,3.0,11.0


# Total games percentage

### Partidas totales ganadas

In [8]:
rapid_win = stats['Rapid_win'].tolist()
rapid_win[:5]

[nan, 12.0, nan, 43.0, 122.0]

In [9]:
blitz_win = stats['Blitz_win'].tolist()
blitz_win[:5]

[nan, 976.0, nan, 1402.0, 1012.0]

In [10]:
total_wins = [x + y for x, y in zip(rapid_win, blitz_win)]
total_wins[:5]

[nan, 988.0, nan, 1445.0, 1134.0]

### Partidas totales empatadas

In [11]:
rapid_draw = stats['Rapid_draw'].tolist()
rapid_draw[:5]

[nan, 5.0, nan, 23.0, 103.0]

In [12]:
blitz_draw = stats['Blitz_draw'].tolist()
blitz_draw[:5]

[nan, 280.0, nan, 292.0, 194.0]

In [13]:
total_draws = [x + y for x, y in zip(rapid_draw, blitz_draw)]
total_draws[:5]

[nan, 285.0, nan, 315.0, 297.0]

### Partidas totales perdidas

In [14]:
rapid_loss = stats['Rapid_loss'].tolist()
rapid_loss[:5]

[nan, 6.0, nan, 8.0, 46.0]

In [15]:
blitz_loss = stats['Blitz_loss'].tolist()
blitz_loss[:5]

[nan, 847.0, nan, 334.0, 322.0]

In [16]:
total_losses = [x + y for x, y in zip(rapid_loss, blitz_loss)]
total_losses[:5]

[nan, 853.0, nan, 342.0, 368.0]

# Total_games

In [17]:
total_games = [x + y + z for x, y, z in zip(total_wins, total_draws, total_losses)]
total_games[:5]

[nan, 2126.0, nan, 2102.0, 1799.0]

### % wins_online

In [18]:
wins_online = [round((x / y)*100, 2) for x, y in zip(total_wins, total_games)]
wins_online[:5]

[nan, 46.47, nan, 68.74, 63.04]

### % draws_online

In [19]:
draws_online = [round((x / y)*100, 2) for x, y in zip(total_draws, total_games)]
draws_online[:5]

[nan, 13.41, nan, 14.99, 16.51]

### % losses_online

In [20]:
losses_online = [round((x / y)*100, 2) for x, y in zip(total_losses, total_games)]
losses_online[:5]

[nan, 40.12, nan, 16.27, 20.46]

In [21]:
stats['wins_online'] = wins_online

In [22]:
stats['draws_online'] = draws_online

In [23]:
stats['losses_online'] = losses_online

In [24]:
stats.columns = stats.columns.str.capitalize()
stats

,Name,Country,Current_rapid_rating,Highest_rapid_rating,Rapid_win,Rapid_draw,Rapid_loss,Current_blitz_rating,Highest_blitz_rating,Blitz_win,Blitz_draw,Blitz_loss,Wins_online,Draws_online,Losses_online
0,"Engine, Komodo",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Plát, Vojtěch",Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0,46.47,13.41,40.12
2,"Carlsen, PlayMagnus",Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Carlsen, Magnus",Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,68.74,14.99,16.27
4,"Caruana, Fabiano",United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,63.04,16.51,20.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,"Zvonko, Stanojoski",North Macedonia,NaN,NaN,NaN,NaN,NaN,2375.0,2430.0,50.0,9.0,27.0,NaN,NaN,NaN
1504,"Gurevich, Dmitry",United States,2375.0,2555.0,7.0,2.0,6.0,2415.0,2675.0,3293.0,538.0,2521.0,51.83,8.48,39.69
1505,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2961.0,2967.0,338.0,22.0,96.0,NaN,NaN,NaN
1506,"Zubov, Alexander",Ukraine,NaN,NaN,NaN,NaN,NaN,2771.0,2763.0,138.0,3.0,11.0,NaN,NaN,NaN


# what do i do with the nulls?

In [25]:
stats.describe()

,Current_rapid_rating,Highest_rapid_rating,Rapid_win,Rapid_draw,Rapid_loss,Current_blitz_rating,Highest_blitz_rating,Blitz_win,Blitz_draw,Blitz_loss,Wins_online,Draws_online,Losses_online
count,1055.000000,997.000000,1055.000000,1055.000000,1055.000000,1388.000000,1382.000000,1388.000000,1388.000000,1388.000000,1025.000000,1025.000000,1025.000000
mean,2382.517536,2555.735206,35.672986,12.870142,15.783886,2696.157781,2797.417511,1460.873199,272.278098,1046.095821,55.130829,10.832410,34.036615
std,316.522940,253.590732,89.034619,41.688089,43.302878,242.827932,222.382485,2760.901696,463.509412,2049.376713,8.745193,4.579518,8.470130
min,263.000000,1200.000000,0.000000,0.000000,0.000000,255.000000,1265.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000
25%,2306.000000,2470.000000,4.000000,1.000000,2.000000,2565.000000,2680.250000,151.000000,23.000000,79.000000,49.590000,8.140000,29.260000
50%,2455.000000,2590.000000,12.000000,4.000000,6.000000,2725.000000,2824.500000,564.500000,100.500000,344.000000,53.720000,10.030000,35.490000
75%,2567.500000,2715.000000,31.000000,12.000000,16.000000,2860.250000,2948.000000,1710.500000,334.250000,1223.250000,59.140000,12.680000,39.890000
max,2948.000000,3045.000000,1561.000000,1150.000000,878.000000,3287.000000,3332.000000,39863.000000,5277.000000,31979.000000,100.000000,50.000000,60.000000


In [26]:
stats.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Name                  1241 non-null   object 
 1   Country               1508 non-null   object 
 2   Current_rapid_rating  1055 non-null   float64
 3   Highest_rapid_rating  997 non-null    float64
 4   Rapid_win             1055 non-null   float64
 5   Rapid_draw            1055 non-null   float64
 6   Rapid_loss            1055 non-null   float64
 7   Current_blitz_rating  1388 non-null   float64
 8   Highest_blitz_rating  1382 non-null   float64
 9   Blitz_win             1388 non-null   float64
 10  Blitz_draw            1388 non-null   float64
 11  Blitz_loss            1388 non-null   float64
 12  Wins_online           1025 non-null   float64
 13  Draws_online          1025 non-null   float64
 14  Losses_online         1025 non-null   float64
dtypes: float64(13), objec

In [27]:
stats = stats.to_csv(r'..\data\stats_chess_com.csv', index=False)

# df = top_100_scraping = draws/wins/losses (live torunaments)'yotachess'

In [28]:
df_original = pd.read_csv(r'..\data\top_100_scraping.csv')
df = df_original.copy()
df.head()

,name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",NaN,NaN,NaN
3,"Ding, Liren",760 (33.48 %),413 (47.4 %),434 (19.12 %)
4,"Firouzja, Alireza",1484 (52.68 %),223 (23.82 %),662 (23.5 %)


In [29]:
df.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    100 non-null    object
 1   Wins    71 non-null     object
 2   Draws   71 non-null     object
 3   Losses  71 non-null     object
dtypes: object(4)
memory usage: 24.6 KB


# table = rating_top_100 = actual rating in all formats (oficial elo)'2700Chess'

In [30]:
table_original = pd.read_csv(r'..\data\rating_top_100.csv')
table = table_original.copy()
table.head()

,#,Name,Classic,Rapid,Blitz,Age
0,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32
1,2,"Caruana, Fabiano",2782.6,2767.4,2813.0,31
2,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35
3,4,"Ding, Liren",2780.0,2830.0,2787 i,30
4,5,"Firouzja, Alireza",2777.0,2742.0,2896.0,20


In [31]:
table.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   #        100 non-null    int64  
 1   Name     100 non-null    object 
 2   Classic  100 non-null    float64
 3   Rapid    100 non-null    object 
 4   Blitz    100 non-null    object 
 5   Age      100 non-null    int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 21.9 KB


# data = scraping_top_100 = draws/wins/losses (better scraping, two more rows than df, 'wtf?')

In [32]:
data_original = pd.read_csv(r'..\data\escrapeo_top_100.csv')
data = data_original.copy()
data.head()

,Jugador,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",NaN,NaN,NaN
3,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
4,"Ding, Liren",NaN,NaN,NaN


In [33]:
# renombro las columnas
data.columns = ['name', 'Wins', 'Draws', 'Losses']
data

,name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",NaN,NaN,NaN
3,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
4,"Ding, Liren",NaN,NaN,NaN
...,...,...,...,...
97,"Najer, Evgeniy",1181 (43.18 %),None (34.52 %),610 (22.3 %)
98,"Saric, Ivan",1153 (43.31 %),None (36.03 %),550 (20.66 %)
99,"Malakhov, Vladimir",919 (41.03 %),None (44.29 %),329 (14.69 %)
100,"Liang, Awonder",530 (44.17 %),None (29.0 %),322 (26.83 %)


In [34]:
data.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    102 non-null    object
 1   Wins    93 non-null     object
 2   Draws   93 non-null     object
 3   Losses  93 non-null     object
dtypes: object(4)
memory usage: 27.4 KB


# complete data

In [35]:
# rows with 'NaN' values
Wins_filtradas = data[data.isnull().any(axis=1)]
Wins_filtradas

,name,Wins,Draws,Losses
2,"Nakamura, Hikaru",NaN,NaN,NaN
4,"Ding, Liren",NaN,NaN,NaN
5,"Firouzja, Alireza",NaN,NaN,NaN
16,"Mamedyarov, Shakhriyar",NaN,NaN,NaN
18,"Le, Quang Liem",NaN,NaN,NaN
19,"Grischuk, Alexander",NaN,NaN,NaN
22,"Topalov, Veselin",NaN,NaN,NaN
23,"Wei, Yi",NaN,NaN,NaN
25,"Vitiugov, Nikita",NaN,NaN,NaN


In [36]:
# list of players with 'NaN'
lst = list(Wins_filtradas.name)
lst

['Nakamura, Hikaru',
 'Ding, Liren',
 'Firouzja, Alireza',
 'Mamedyarov, Shakhriyar',
 'Le, Quang Liem',
 'Grischuk, Alexander',
 'Topalov, Veselin',
 'Wei, Yi',
 'Vitiugov, Nikita']

In [37]:
nombres_filtrados = []
for n in lst:
    filtro = df.loc[df['name'] == n]
    nombres_filtrados.append(filtro)
    
nombres_filtrados

[               name Wins Draws Losses
 2  Nakamura, Hikaru  NaN   NaN    NaN,
           name           Wins         Draws         Losses
 3  Ding, Liren  760 (33.48 %)  413 (47.4 %)  434 (19.12 %),
                 name            Wins          Draws        Losses
 4  Firouzja, Alireza  1484 (52.68 %)  223 (23.82 %)  662 (23.5 %),
                       name            Wins          Draws          Losses
 15  Mamedyarov, Shakhriyar  2034 (41.27 %)  727 (37.32 %)  1055 (21.41 %),
               name Wins Draws Losses
 16  Le, Quang Liem  NaN   NaN    NaN,
                    name            Wins          Draws        Losses
 17  Grischuk, Alexander  2066 (39.41 %)  805 (41.68 %)  991 (18.9 %),
                 name            Wins          Draws         Losses
 20  Topalov, Veselin  1002 (33.71 %)  None (42.8 %)  698 (23.49 %),
        name           Wins           Draws         Losses
 21  Wei, Yi  704 (39.33 %)  None (41.62 %)  341 (19.05 %),
                 name           Wins    

In [38]:
type(nombres_filtrados)

list

In [39]:
nombres_filtrados[0:2]

[               name Wins Draws Losses
 2  Nakamura, Hikaru  NaN   NaN    NaN,
           name           Wins         Draws         Losses
 3  Ding, Liren  760 (33.48 %)  413 (47.4 %)  434 (19.12 %)]

In [40]:
# convertir la lista de nombres con los valores que faltaban en un dataframe
data_filter = [
    {'name': 'Nakamura, Hikaru', 'Wins': None, 'Draws': None, 'Losses': None},
    {'name': 'Ding, Liren', 'Wins': '760 (33.48 %)', 'Draws': '413 (47.4 %)', 'Losses': '434 (19.12 %)'},
    {'name': 'Firouzja, Alireza', 'Wins': '1484 (52.68 %)', 'Draws': '223 (23.82 %)', 'Losses': '662 (23.5 %)'},
    {'name': 'Mamedyarov, Shakhriyar', 'Wins': '2034 (41.27 %)', 'Draws': '727 (37.32 %)', 'Losses': '1055 (21.41 %)'},
    {'name': 'Le, Quang Liem', 'Wins': None, 'Draws': None, 'Losses': None},
    {'name': 'Grischuk, Alexander', 'Wins': '2066 (39.41 %)', 'Draws': '805 (41.68 %)', 'Losses': '991 (18.9 %)'},
    {'name': 'Topalov, Veselin', 'Wins': '1002 (33.71 %)', 'Draws': 'None (42.8 %)', 'Losses': '698 (23.49 %)'},
    {'name': 'Wei, Yi', 'Wins': '704 (39.33 %)', 'Draws': 'None (41.62 %)', 'Losses': '341 (19.05 %)'},
    {'name': 'Vitiugov, Nikita', 'Wins': '898 (39.27 %)', 'Draws': 'None (45.65 %)', 'Losses': '345 (15.09 %)'}
]

# Crear el DataFrame
filter_name = pd.DataFrame(data_filter)

# Imprimir el DataFrame
print(filter_name)

                     name            Wins           Draws          Losses
0        Nakamura, Hikaru            None            None            None
1             Ding, Liren   760 (33.48 %)    413 (47.4 %)   434 (19.12 %)
2       Firouzja, Alireza  1484 (52.68 %)   223 (23.82 %)    662 (23.5 %)
3  Mamedyarov, Shakhriyar  2034 (41.27 %)   727 (37.32 %)  1055 (21.41 %)
4          Le, Quang Liem            None            None            None
5     Grischuk, Alexander  2066 (39.41 %)   805 (41.68 %)    991 (18.9 %)
6        Topalov, Veselin  1002 (33.71 %)   None (42.8 %)   698 (23.49 %)
7                 Wei, Yi   704 (39.33 %)  None (41.62 %)   341 (19.05 %)
8        Vitiugov, Nikita   898 (39.27 %)  None (45.65 %)   345 (15.09 %)


In [41]:
# Fill the null values with the actual value
data[['Wins', 'Draws', 'Losses']] = data[['Wins', 'Draws', 'Losses']].combine_first(df[['Wins', 'Draws', 'Losses']])
data.head(20)

,name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",NaN,NaN,NaN
3,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
4,"Ding, Liren",1484 (52.68 %),223 (23.82 %),662 (23.5 %)
5,"Firouzja, Alireza",1566 (40.26 %),589 (39.02 %),806 (20.72 %)
6,"Nepomniachtchi, Ian",1566 (40.26 %),589 (39.02 %),806 (20.72 %)
7,"So, Wesley",1588 (39.11 %),586 (43.79 %),694 (17.09 %)
8,"Giri, Anish",1226 (32.51 %),688 (48.02 %),734 (19.46 %)
9,Gukesh D,572 (30.94 %),None (22.01 %),870 (47.05 %)


In [42]:
data.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    102 non-null    object
 1   Wins    100 non-null    object
 2   Draws   100 non-null    object
 3   Losses  100 non-null    object
dtypes: object(4)
memory usage: 28.1 KB


# inspect the two weird rows with nulls

In [43]:
# Detectar las filas con valores nulos
filas_con_nulos = data[data.isnull().any(axis=1)]
filas_con_nulos

,name,Wins,Draws,Losses
2,"Nakamura, Hikaru",NaN,NaN,NaN
16,"Mamedyarov, Shakhriyar",NaN,NaN,NaN


In [44]:
data.name.value_counts()

Nakamura, Hikaru              2
Mamedyarov, Shakhriyar        2
Carlsen, Magnus               1
Wojtaszek, Radoslaw           1
Adams, Michael                1
                             ..
Sjugirov, Sanan               1
Maghsoodloo, Parham           1
Dubov, Daniil                 1
Harikrishna, Pentala          1
Aravindh, Chithambaram VR.    1
Name: name, Length: 100, dtype: int64

In [45]:
data = data.dropna()
data

,name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
3,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
4,"Ding, Liren",1484 (52.68 %),223 (23.82 %),662 (23.5 %)
5,"Firouzja, Alireza",1566 (40.26 %),589 (39.02 %),806 (20.72 %)
...,...,...,...,...
97,"Najer, Evgeniy",1181 (43.18 %),None (34.52 %),610 (22.3 %)
98,"Saric, Ivan",1153 (43.31 %),None (36.03 %),550 (20.66 %)
99,"Malakhov, Vladimir",919 (41.03 %),None (44.29 %),329 (14.69 %)
100,"Liang, Awonder",530 (44.17 %),None (29.0 %),322 (26.83 %)


In [46]:
data = data.reset_index(drop = True)
data

,name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
3,"Ding, Liren",1484 (52.68 %),223 (23.82 %),662 (23.5 %)
4,"Firouzja, Alireza",1566 (40.26 %),589 (39.02 %),806 (20.72 %)
...,...,...,...,...
95,"Najer, Evgeniy",1181 (43.18 %),None (34.52 %),610 (22.3 %)
96,"Saric, Ivan",1153 (43.31 %),None (36.03 %),550 (20.66 %)
97,"Malakhov, Vladimir",919 (41.03 %),None (44.29 %),329 (14.69 %)
98,"Liang, Awonder",530 (44.17 %),None (29.0 %),322 (26.83 %)


In [47]:
data.columns = ['Name', 'Wins', 'Draws', 'Losses']
data

,Name,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",3953 (51.91 %),1107 (31.9 %),1233 (16.19 %)
3,"Ding, Liren",1484 (52.68 %),223 (23.82 %),662 (23.5 %)
4,"Firouzja, Alireza",1566 (40.26 %),589 (39.02 %),806 (20.72 %)
...,...,...,...,...
95,"Najer, Evgeniy",1181 (43.18 %),None (34.52 %),610 (22.3 %)
96,"Saric, Ivan",1153 (43.31 %),None (36.03 %),550 (20.66 %)
97,"Malakhov, Vladimir",919 (41.03 %),None (44.29 %),329 (14.69 %)
98,"Liang, Awonder",530 (44.17 %),None (29.0 %),322 (26.83 %)


In [48]:
data.to_csv(r'..\data\top_100_wdl.csv', index=False)

In [49]:
import re

def extraer_porcentaje(cadena):
    patron = r'\((\d+\.\d+)\s%\)'
    resultado = re.search(patron, cadena)
    
    if resultado:
        porcentaje = resultado.group(1)
        return porcentaje
    
    return None


In [50]:
data['Wins'] = data['Wins'].apply(extraer_porcentaje)
data['Wins']

0     48.53
1     42.75
2     51.91
3     52.68
4     40.26
      ...  
95    43.18
96    43.31
97    41.03
98    44.17
99      8.7
Name: Wins, Length: 100, dtype: object

In [51]:
data['Draws'] = data['Draws'].apply(extraer_porcentaje)
data['Draws']

0     35.27
1     34.49
2      31.9
3     23.82
4     39.02
      ...  
95    34.52
96    36.03
97    44.29
98     29.0
99    23.28
Name: Draws, Length: 100, dtype: object

In [52]:
data['Losses'] = data['Losses'].apply(extraer_porcentaje)
data['Losses']

0      16.2
1     22.76
2     16.19
3      23.5
4     20.72
      ...  
95     22.3
96    20.66
97    14.69
98    26.83
99    68.02
Name: Losses, Length: 100, dtype: object

# Merge dataframes(final_dataframe)

In [53]:
stats_live_tournaments = pd.merge(table, data, on='Name', how='left')
stats_live_tournaments

,#,Name,Classic,Rapid,Blitz,Age,Wins,Draws,Losses
0,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32,48.53,35.27,16.2
1,2,"Caruana, Fabiano",2782.6,2767.4,2813.0,31,42.75,34.49,22.76
2,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35,51.91,31.9,16.19
3,4,"Ding, Liren",2780.0,2830.0,2787 i,30,52.68,23.82,23.5
4,5,"Firouzja, Alireza",2777.0,2742.0,2896.0,20,40.26,39.02,20.72
...,...,...,...,...,...,...,...,...,...
95,96,"Najer, Evgeniy",2648.0,2579.0,2545.0,46,43.18,34.52,22.3
96,97,"Saric, Ivan",2647.5,2631.0,2586.0,32,43.31,36.03,20.66
97,98,"Malakhov, Vladimir",2646.0,2666.0,2652.0,42,41.03,44.29,14.69
98,99,"Liang, Awonder",2645.6,2518.4,2538.0,20,44.17,29.0,26.83


In [54]:
# cambiar nombres de las columnas para no confundirme a la hora de contrstar los datos lt = live tournaments
stats_live_tournaments.columns = ['#', 'Name', 'Classic', 'Rapid', 'Blitz', 'Age', '% Wins_lt', '% Draws_lt', '% Losses_lt']
stats_live_tournaments

,#,Name,Classic,Rapid,Blitz,Age,% Wins_lt,% Draws_lt,% Losses_lt
0,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32,48.53,35.27,16.2
1,2,"Caruana, Fabiano",2782.6,2767.4,2813.0,31,42.75,34.49,22.76
2,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35,51.91,31.9,16.19
3,4,"Ding, Liren",2780.0,2830.0,2787 i,30,52.68,23.82,23.5
4,5,"Firouzja, Alireza",2777.0,2742.0,2896.0,20,40.26,39.02,20.72
...,...,...,...,...,...,...,...,...,...
95,96,"Najer, Evgeniy",2648.0,2579.0,2545.0,46,43.18,34.52,22.3
96,97,"Saric, Ivan",2647.5,2631.0,2586.0,32,43.31,36.03,20.66
97,98,"Malakhov, Vladimir",2646.0,2666.0,2652.0,42,41.03,44.29,14.69
98,99,"Liang, Awonder",2645.6,2518.4,2538.0,20,44.17,29.0,26.83


In [56]:
stats_live_tournaments = stats_live_tournaments.to_csv(r'..\data\stats_live_tournaments.csv', index=False)